In [1]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

In [2]:
shape = Box((-1,-1,-1), (1,1,1)) # create a box
geo = OCCGeometry(shape) # create a geometry from the box
mesh = Mesh(geo.GenerateMesh(maxh=0.2)) # create a mesh from the geometry
#Draw (mesh); # draw the mesh

In [3]:
order = 2
Hcc = HCurlCurl(mesh, order=order ,  dirichlet=".*") # for the gamma variable
Hdd = HDivDiv(mesh, order=order ) # , orderinner=order+1) fr t
# fesalpha = H1(mesh, order=order+1) # , dirichlet=".*")
Hc = HCurl(mesh, order=order) # for the div of a Hdd field

k , dk= Hdd.TnT() 
p, dp = Hc.TnT()
g, dg = Hcc.TnT()

In [4]:
n = specialcf.normal(3)
t = specialcf.tangential(3, True)
bbndtang  = specialcf.EdgeFaceTangentialVectors(3)
tef1 = bbndtang[:,0]
tef2 = bbndtang[:,1]
nu1 = Cross(tef1,t)
nu2 = Cross(tef2,t)

def IncOp(g, dg):
    return InnerProduct(curl(g), curl(dg).trans)*dx \
        + (curl(g)*n) * Cross (dg*n, n) * dx(element_vb=BND) \
        + (curl(dg)*n) * Cross (g*n, n) * dx(element_vb=BND) \
        + (g[nu1,t]*dg[t,tef1]-g[nu2,t]*dg[t,tef2])*dx(element_vb=BBND)

# print ( (gamma*nu1*t)*(dgamma*t*tef1))

def J(g): return g - 0.5*Trace(g)*Id(3)

def Constraint(k, p):
    return  div( k)*p * dx - InnerProduct(k*n - k[n,n]*n , p - (p*n)*n ) * dx(element_boundary = True)

def Curl(u):
    if u.dim == 3:
        return CF( (u[1].Diff(z)- u[2].Diff(y), u[2].Diff(x)- u[0].Diff(z), u[0].Diff(y)- u[1].Diff(x)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )

def Transpose(u):
    return CF( (u[0,0] , u[1,0] , u[2,0] , u[0,1] , u[1,1] , u[2,1] , u[0,2] , u[1,2] , u[2,2] ),dims=(3,3) )

def Symetric(u): return u + Transpose(u) 

In [7]:

# 3 minutes for a cube of size 2 and a mesh of size 0.2 and order 3
with TaskManager():
    inv_PROJ = PROJ.mat.Inverse(inverse="sparsecholesky", freedofs=fes.FreeDofs())
    inv_Mcc = Mcc.mat.Inverse(inverse="sparsecholesky", freedofs=fes.FreeDofs())
    inv_Mdd = Mdd.mat.Inverse(inverse="sparsecholesky", freedofs=fes.FreeDofs())


In [5]:
gf_g = GridFunction(Hcc)
gf_inc_g = GridFunction(Hcc)

peak = exp(-10*(x*x+y*y+z*z))/1000
cf_gamma = CF((peak,0,0, 0,0,0,  0,0,0), dims=(3,3))
cf_symcurl_gamma = Symetric(Curl(cf_gamma))

#print(" total magnitude")
#Draw(InnerProduct(cf_symcurl_gamma,cf_symcurl_gamma), mesh, "SCGamma", clipping={"x": -1, "y": 0, "z": 0})
#print(" (x,x) component")
#Draw(InnerProduct(cf_symcurl_gamma[0,0],cf_symcurl_gamma[0,0]), mesh, "SCGamma", clipping={"x": -1, "y": 0, "z": 0})
#print(" (x,y) component")
#Draw(InnerProduct(cf_symcurl_gamma[0,1],cf_symcurl_gamma[0,1]), mesh, "SCGamma", clipping={"x": -1, "y": 0, "z": 0})
#print(" (x,z) component")
#Draw(InnerProduct(cf_symcurl_gamma[0,2],cf_symcurl_gamma[0,2]), mesh, "SCGamma", clipping={"x": -1, "y": 0, "z": 0})
#print(" (y,y) component")
#Draw(InnerProduct(cf_symcurl_gamma[1,1],cf_symcurl_gamma[1,1]), mesh, "SCGamma", clipping={"x": -1, "y": 0, "z": 0})
#print(" (y,z) component")
#Draw(InnerProduct(cf_symcurl_gamma[1,2],cf_symcurl_gamma[1,2]), mesh, "SCGamma", clipping={"x": -1, "y": 0, "z": 0})
#print(" (z,z) component")
#Draw(InnerProduct(cf_symcurl_gamma[2,2],cf_symcurl_gamma[2,2]), mesh, "SCGamma", clipping={"x": -1, "y": 0, "z": 0})







In [6]:

gf_g.Set( cf_symcurl_gamma, dual=True, bonus_intorder=12)




In [7]:
print ("gamma\n")
scene1 = Draw (InnerProduct(gf_g,gf_g), mesh, draw_surf=False, clipping={"x": -1, "y": 0, "z": 0})
#print ("inc gamma\n")
#scene2 = Draw (InnerProduct(gf_inc_g,gf_inc_g), mesh, draw_surf=False, clipping={"x": -1, "y": 0, "z": 0})
#print ("kappa \n")
#scene3 = Draw (InnerProduct(gf_k, gf_k), mesh, draw_surf=False, clipping={"x": -1, "y": 0, "z": 0})
#print ("J kappa\n")
#scene4 = Draw (InnerProduct(gf_J_k, gf_J_k), mesh, draw_surf=False, clipping={"x": -1, "y": 0, "z": 0})


gamma



WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

In [8]:
peak = exp(-10*(x*x+y*y+z*z))/1000
cf_gamma = CF((peak,0,0, 0,0,0,  0,0,0), dims=(3,3))
cf_symcurl_gamma = Symetric(Curl(cf_gamma))

In [9]:
X = mesh(0,0.2,0)
print(cf_symcurl_gamma(X))

(0.0, 9.302559362207019e-20, -0.0026812801841425565, 9.302559362207019e-20, 0.0, 0.0, -0.0026812801841425565, 0.0, 0.0)
